In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/chriszapp/datasets/main/books.csv', on_bad_lines='skip')

In [ ]:
df.columns

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')

In [ ]:
!pip install dash
!pip install dash_bootstrap_components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.2/221.2 kB 2.7 MB/s eta 0:00:00


#First part

In [ ]:
from dash import Dash, html
import dash_bootstrap_components as dbc
from dash import html, dcc
import plotly.express as px

import pandas as pd


df = pd.read_csv('https://raw.githubusercontent.com/chriszapp/datasets/main/books.csv', on_bad_lines='skip')
app = Dash(__name__, external_stylesheets=[dbc.themes.QUARTZ])
books = px.bar(df.loc[:10], x = 'title', y = '  num_pages')
books.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
books.update_yaxes(title_font_color="white", color="white", title_font_size=14)
books.update_xaxes(title_font_color="white", color="white", title_font_size=14,
                   tickangle=45, tickvals = df['title'][:10],
                   ticktext= [i[:15] for i in df.loc[:20, 'title'] ])


# Layout de base
app.layout = html.Div([
    html.H1("Books", style={'textAlign': 'center'}),
    dbc.Row([dbc.Col([dbc.Row(dcc.Input(id='author', type='text', placeholder='Authors name'), style={'margin-top': '50px', 'margin-bottom' : '50px'}),
                      dbc.Row(dcc.Input(id='pages', type="number", min=0, max=1000, step=1, placeholder='Maximal number of pages'))]),
        dbc.Col(dcc.Graph(id='books', figure=books), style={'flex' : '2'})])
])

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

#Second part

In [ ]:
from dash import Dash, html
import dash_bootstrap_components as dbc
from dash import html, dcc
from dash.dependencies import Input, Output

import plotly.express as px

import pandas as pd


app = Dash(__name__, external_stylesheets=[dbc.themes.QUARTZ])


app.layout = html.Div([
    html.H1("Livres", style={'textAlign': 'center'}),
    dbc.Row([dbc.Col([dbc.Row(dcc.Input(id='author', type='text', placeholder='Authors name'),
                              style={'margin-top': '50px', 'margin-bottom' : '50px'}),
                      dbc.Row(dcc.Input(id='pages', type="number",
                                        min=0, max=7000, step=1,
                                        placeholder='Maximal number of pages'))], style={'margin-left' : '50px'}),
        dbc.Col(dcc.Graph(id='books'), style={'flex' : '2'})])
])


@app.callback(Output('books', 'figure'),
              [Input('author', 'value'),
               Input('pages', 'value')])
def update_graph(value_autor, value_pages):
    df = pd.read_csv('https://raw.githubusercontent.com/chriszapp/datasets/main/books.csv', on_bad_lines='skip')

    if value_autor is not None:
        df = df[df['authors'].str.contains(value_autor, case=False)].reset_index()
    if value_pages is not None:
        df = df[df['  num_pages']>=value_pages].reset_index()

    books = px.bar(df.loc[:10], x = 'title', y = '  num_pages')


    books.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    books.update_yaxes(title_font_color="white", color="white", title_font_size=14)
    books.update_xaxes(title_font_color="white", color="white", title_font_size=14,
                   tickangle=45, tickvals = df['title'][:10],
                   ticktext= [i[:15] for i in df.loc[:10, 'title'] ])
    return books

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

In [ ]:
df['language_code'].value_counts()

eng      8908
en-US    1408
spa       218
en-GB     214
fre       144
ger        99
jpn        46
mul        19
zho        14
grc        11
por        10
en-CA       7
ita         5
enm         3
lat         3
swe         2
rus         2
srp         1
nl          1
msa         1
glg         1
wel         1
ara         1
nor         1
tur         1
gla         1
ale         1
Name: language_code, dtype: int64

In [ ]:
len(df.publisher.unique())

2290

In [ ]:
df['publication_date'][11094] = '6/30/1982'

<ipython-input-8-b079fe329cfb>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df['publication_date'] = pd.to_datetime(df['publication_date'])

ValueError: day is out of range for month present at position 8177

In [ ]:
language = 'fre'
df[(df['language_code']==language) & (df['ratings_count']>df['ratings_count'].median())].groupby('authors').mean().sort_values(by='average_rating', ascending=False).reset_index()[['authors', 'average_rating']]

In [ ]:
df[df['ratings_count']>df['ratings_count'].median()].sort_values(by='average_rating', ascending=False)

In [ ]:
import plotly.express as px


def filtre(df, lang, typ):
  if lang is not None:
    if typ == 'author':
      df = df[(df['language_code']==lang) & (df['ratings_count']>df['ratings_count'].median())].groupby('authors').mean().sort_values(by='average_rating', ascending=False).reset_index()[['authors', 'average_rating']]
    if typ == 'book':
      df = df[(df['language_code']==lang) & (df['ratings_count']>df['ratings_count'].median())].sort_values(by='average_rating', ascending=False).reset_index()[['title', 'average_rating']]
  else:
    df[df['ratings_count']>df['ratings_count'].median()].sort_values(by='average_rating', ascending=False)[['title', 'average_rating']]

  return df




In [ ]:
to_viz = filtre(df, 'fre', 'author')

In [ ]:
to_viz
px.bar(to_viz, x = 'authors', y = 'average_rating')